In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
Train_data=pd.read_csv('Data/train.csv')
Test_data=pd.read_csv('Data/test.csv')

In [3]:
print('Train Data Shape : {} and That of Test Data Shape: {}'.format(Train_data.shape, Test_data.shape))

Train Data Shape : (233154, 41) and That of Test Data Shape: (112392, 40)


In [4]:
Train=Train_data.copy()
Test=Test_data.copy()

In [0]:
Train.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID', 'Date.of.Birth',
       'Employment.Type', 'DisbursalDate', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES',
       'loan_default'],
      dtype='object')

In [5]:
Train['Date.of.Birth']= pd.to_datetime(Train['Date.of.Birth'],format='%d-%m-%y')
Train['Date.of.Birth'] = Train['Date.of.Birth'].mask(Train['Date.of.Birth'].dt.year > 2018, Train['Date.of.Birth'] - pd.offsets.DateOffset(years=100))
Test['Date.of.Birth']= pd.to_datetime(Test['Date.of.Birth'],format='%d-%m-%y')
Test['Date.of.Birth'] = Test['Date.of.Birth'].mask(Test['Date.of.Birth'].dt.year > 2018, Test['Date.of.Birth'] - pd.offsets.DateOffset(years=100))

In [6]:
Train['DisbursalDate'] = pd.to_datetime(Train['DisbursalDate'], format='%d-%m-%y')
Test['DisbursalDate'] = pd.to_datetime(Test['DisbursalDate'], format='%d-%m-%y')

In [7]:
tr_ag=Train['DisbursalDate'] -Train['Date.of.Birth']
Train['Age']=tr_ag//np.timedelta64(1,'Y')
te_ag=Test['DisbursalDate'] - Test['Date.of.Birth']
Test['Age']=te_ag//np.timedelta64(1,'Y')

In [13]:
Train.drop(index=210252, inplace=True)

In [16]:
Train.shape

(233153, 38)

In [17]:
y=Train.loan_default

In [0]:
Train.asset_cost.sort_values()

231111      37000
231112      37000
167537      37129
124403      37230
207575      37310
208968      37377
70867       37658
192454      37816
213021      38055
213062      38055
213065      38059
213015      38063
162117      38098
162138      38098
212539      38116
212542      38116
212562      38241
212556      38241
212537      38241
212580      38241
212551      38241
50100       38500
53662       38620
133755      38650
211423      38710
211425      38710
202773      38738
114717      38752
114746      38752
114661      38752
           ...   
195926     247018
133030     247045
193778     247078
94274      251767
187033     252140
53737      252434
193780     253442
94368      253589
94317      254177
115043     255315
113758     255398
195962     256955
167303     258214
106002     259000
92942      259409
227586     260715
46219      264500
173619     277600
100889     280100
168864     281164
173633     282600
173597     286350
210251     378092
210250     378092
210254    

In [8]:
drop_cols=['Date.of.Birth', 'DisbursalDate', 'UniqueID','MobileNo_Avl_Flag']

In [9]:
Train.drop(drop_cols, axis=1, inplace=True)
Test.drop(drop_cols, axis=1, inplace=True)

In [10]:
def conv_month(Yr_Mnth):
  import re
  lst=re.findall('\d', Yr_Mnth)
  return 12*int(lst[0]) + int(lst[1])

In [11]:
Train['AVERAGE.ACCT.AGE']= Train['AVERAGE.ACCT.AGE'].apply(conv_month)
Train['CREDIT.HISTORY.LENGTH']= Train['CREDIT.HISTORY.LENGTH'].apply(conv_month)
Test['AVERAGE.ACCT.AGE']= Test['AVERAGE.ACCT.AGE'].apply(conv_month)
Test['CREDIT.HISTORY.LENGTH']= Test['CREDIT.HISTORY.LENGTH'].apply(conv_month)

In [12]:
Train['Employment.Type'].fillna('No_Emp', inplace=True)
Test['Employment.Type'].fillna('No_Emp', inplace=True)

In [18]:
Obj_Cols=['Employment.Type', 'PERFORM_CNS.SCORE.DESCRIPTION']

In [19]:
Tr_Ob_Attr=Train[Obj_Cols]
Te_Ob_Attr=Test[Obj_Cols]

In [20]:
Tr_Ob_Attr.head()

,Employment.Type,PERFORM_CNS.SCORE.DESCRIPTION
0,Salaried,No Bureau History Available
1,Self employed,I-Medium Risk
2,Self employed,No Bureau History Available
3,Self employed,L-Very High Risk
4,Self employed,No Bureau History Available


In [21]:
Tr_Ob_Attr['PERFORM_CNS.SCORE.DESCRIPTION']=Tr_Ob_Attr['PERFORM_CNS.SCORE.DESCRIPTION'].map({'No Bureau History Available' : 0,  'I-Medium Risk' :  1, 'L-Very High Risk' :  2,  'A-Very Low Risk':  3,  'Not Scored: Not Enough Info available on the customer' : 4,  'D-Very Low Risk' : 5 ,  'M-Very High Risk' : 6,  'B-Very Low Risk' : 7,  'C-Very Low Risk' : 8,  'E-Low Risk' : 9,  'H-Medium Risk' : 10,   
 'F-Low Risk' : 11 ,  'K-High Risk' : 12 ,  'Not Scored: No Activity seen on the customer (Inactive)' : 13,  'Not Scored: Sufficient History Not Available' : 14,  'Not Scored: No Updates available in last 36 months' : 15,   'G-Low Risk' : 16,  'J-High Risk' : 17,  'Not Scored: Only a Guarantor' : 18,  'Not Scored: More than 50 active Accounts found' : 19})

/home/chinu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
Te_Ob_Attr['PERFORM_CNS.SCORE.DESCRIPTION']=Te_Ob_Attr['PERFORM_CNS.SCORE.DESCRIPTION'].map({'No Bureau History Available' : 0,  'I-Medium Risk' :  1, 'L-Very High Risk' :  2,  'A-Very Low Risk':  3,  'Not Scored: Not Enough Info available on the customer' : 4,  'D-Very Low Risk' : 5 ,  'M-Very High Risk' : 6,  'B-Very Low Risk' : 7,  'C-Very Low Risk' : 8,  'E-Low Risk' : 9,  'H-Medium Risk' : 10,   
 'F-Low Risk' : 11 ,  'K-High Risk' : 12 ,  'Not Scored: No Activity seen on the customer (Inactive)' : 13,  'Not Scored: Sufficient History Not Available' : 14,  'Not Scored: No Updates available in last 36 months' : 15,   'G-Low Risk' : 16,  'J-High Risk' : 17,  'Not Scored: Only a Guarantor' : 18,  'Not Scored: More than 50 active Accounts found' : 19})

/home/chinu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
Tr_Ob_Attr['Employment.Type']=Tr_Ob_Attr['Employment.Type'].map({'Salaried' : 0, 'Self employed' : 1,  'No_Emp' : 2})
Te_Ob_Attr['Employment.Type']=Te_Ob_Attr['Employment.Type'].map({'Salaried' : 0, 'Self employed' : 1,  'No_Emp' : 2})

/home/chinu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/chinu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
#  !pip install category_encoders
 import category_encoders as ce 

In [26]:
for x in Obj_Cols:
  ce_bin = ce.BinaryEncoder(cols = [x])
  Te_Ob_Attr=ce_bin.fit_transform(Te_Ob_Attr)

In [27]:
for x in Obj_Cols:
  ce_bin = ce.BinaryEncoder(cols = [x])
  Tr_Ob_Attr=ce_bin.fit_transform(Tr_Ob_Attr)

In [28]:
Train.drop(Obj_Cols, axis=1, inplace=True)
Train.drop('loan_default', axis=1, inplace=True)
Test.drop(Obj_Cols, axis=1, inplace=True)


In [29]:
Te_Ob_Attr.columns

Index(['PERFORM_CNS.SCORE.DESCRIPTION_0', 'PERFORM_CNS.SCORE.DESCRIPTION_1',
       'PERFORM_CNS.SCORE.DESCRIPTION_2', 'PERFORM_CNS.SCORE.DESCRIPTION_3',
       'PERFORM_CNS.SCORE.DESCRIPTION_4', 'PERFORM_CNS.SCORE.DESCRIPTION_5',
       'Employment.Type_0', 'Employment.Type_1', 'Employment.Type_2'],
      dtype='object')

In [30]:
names=['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id','manufacturer_id', 'Current_pincode_ID',  'State_ID', 'Employee_ID', 
 'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 'Driving_flag', 'Passport_flag', 'Perform_CNS_Scr', 'PRI_No_Of_Acct', 
 'PRI_Act_Acct', 'PRI_Overdue_Acct', 'PRI_Current_Bal',  'PRI_Sanction_Amt', 'PRI_Disbusrsed_Amt', 
 'SEC_No_Of_Acct',  'SEC_Act_Acct', 'SEC_Overdue_Acct', 'SEC_Current_Bal',  'SEC_Sanction_Amt', 'SEC_Disbursed_Amt', 
 'PRI_Instal_Amt',  'SEC_Instal_Amt', 'New_Acct_Last_Six_mnth', 'Deliquent_Acct_Last_Six_Mnth', 'AVG_Acct_Age', 'Credit_History_Len', 'No_Of_Inquiry','Age']

In [31]:
from sklearn import preprocessing
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object

#Train Data Standardize
scaled_df_tr = scaler.fit_transform(Train)
scaled_df_tr = pd.DataFrame(scaled_df_tr, columns=names)

#Test Data Standardize
scaled_df_te = scaler.fit_transform(Test)
scaled_df_te = pd.DataFrame(scaled_df_te, columns=names)

/home/chinu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/chinu/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/chinu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/chinu/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [32]:
Final_Train_All=pd.concat([scaled_df_tr, Tr_Ob_Attr], axis=1)
Final_Test_All=pd.concat([scaled_df_te, Te_Ob_Attr], axis=1)

In [33]:
Final_Train_All.to_csv('Final_Train_All1.csv')
Final_Test_All.to_csv('Final_Test_All1.csv')

In [0]:
Final_Train_All.dropna( inplace=True)

In [34]:
Final_Train_All

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,State_ID,Employee_ID,Aadhar_flag,...,Age,PERFORM_CNS.SCORE.DESCRIPTION_0,PERFORM_CNS.SCORE.DESCRIPTION_1,PERFORM_CNS.SCORE.DESCRIPTION_2,PERFORM_CNS.SCORE.DESCRIPTION_3,PERFORM_CNS.SCORE.DESCRIPTION_4,PERFORM_CNS.SCORE.DESCRIPTION_5,Employment.Type_0,Employment.Type_1,Employment.Type_2
0,-0.294332,-0.935119,1.292129,-0.084998,0.907343,-1.085231,-0.873884,-0.281609,0.459908,0.435917,...,0.006260,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-0.562000,-0.552146,-0.132377,-0.084998,0.907343,-1.085231,-0.846629,-0.281609,0.459908,0.435917,...,-0.095585,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-0.083815,-0.776573,1.299112,-0.084998,0.907343,-1.085231,-0.848863,-0.281609,0.459908,0.435917,...,-0.197431,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.246384,-0.521990,1.198733,-0.084998,0.907343,-1.085231,-0.847076,-0.281609,0.459908,0.435917,...,-1.012195,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,-0.153987,-0.833349,1.190878,-0.084998,0.907343,-1.085231,-0.849756,-0.281609,0.459908,0.435917,...,0.617333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
5,0.012477,-0.747649,1.301731,-0.084998,0.907343,-1.085231,-0.847076,-0.281609,0.459908,0.435917,...,-0.604813,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6,-0.624063,-0.769074,0.146065,-0.084998,0.907343,-1.085231,-0.846629,-0.281609,0.459908,0.435917,...,-0.401122,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7,-0.815478,-0.747649,-0.249340,-0.084998,0.907343,-1.085231,-0.847076,-0.281609,0.459908,0.435917,...,-0.604813,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,-0.049899,-0.743739,1.293002,-0.084998,0.907343,-1.085231,-0.848863,-0.281609,0.459908,0.435917,...,-0.808504,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
9,-0.136444,-0.779787,1.065186,-0.084998,0.907343,-1.085231,-0.851097,-0.281609,0.459908,-2.294015,...,1.635789,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [36]:
Final_Train_All.drop(Final_Train_All.tail(1).index, inplace=True) 

In [37]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [38]:
X=Final_Train_All

In [0]:
Train_data['Employment.Type'].fillna('No_Emp', inplace=True)
Pre_pro_train=Train_data.drop(index=210252, axis=0).dropna()
Pre_pro_train['disbursed_amount'].sort_values()


In [0]:
y=Train_data.loan_default

In [0]:
X.loc[y.index]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,State_ID,Employee_ID,Aadhar_flag,...,Age,PERFORM_CNS.SCORE.DESCRIPTION_0,PERFORM_CNS.SCORE.DESCRIPTION_1,PERFORM_CNS.SCORE.DESCRIPTION_2,PERFORM_CNS.SCORE.DESCRIPTION_3,PERFORM_CNS.SCORE.DESCRIPTION_4,PERFORM_CNS.SCORE.DESCRIPTION_5,Employment.Type_0,Employment.Type_1,Employment.Type_2
0,-0.294332,-0.935119,1.292129,-0.084998,0.907343,-1.085231,-0.873884,-0.281609,0.459908,0.435917,...,0.006260,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-0.562000,-0.552146,-0.132377,-0.084998,0.907343,-1.085231,-0.846629,-0.281609,0.459908,0.435917,...,-0.095585,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-0.083815,-0.776573,1.299112,-0.084998,0.907343,-1.085231,-0.848863,-0.281609,0.459908,0.435917,...,-0.197431,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.246384,-0.521990,1.198733,-0.084998,0.907343,-1.085231,-0.847076,-0.281609,0.459908,0.435917,...,-1.012195,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,-0.153987,-0.833349,1.190878,-0.084998,0.907343,-1.085231,-0.849756,-0.281609,0.459908,0.435917,...,0.617333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
5,0.012477,-0.747649,1.301731,-0.084998,0.907343,-1.085231,-0.847076,-0.281609,0.459908,0.435917,...,-0.604813,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6,-0.624063,-0.769074,0.146065,-0.084998,0.907343,-1.085231,-0.846629,-0.281609,0.459908,0.435917,...,-0.401122,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7,-0.815478,-0.747649,-0.249340,-0.084998,0.907343,-1.085231,-0.847076,-0.281609,0.459908,0.435917,...,-0.604813,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,-0.049899,-0.743739,1.293002,-0.084998,0.907343,-1.085231,-0.848863,-0.281609,0.459908,0.435917,...,-0.808504,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
9,-0.136444,-0.779787,1.065186,-0.084998,0.907343,-1.085231,-0.851097,-0.281609,0.459908,-2.294015,...,1.635789,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [0]:
y.loc[210251]

0

In [39]:
print('Train Shape is  : {} items total, for Test Shape is:  {} true'.format(X.shape, y.shape))

Train Shape is  : (233153, 44) items total, for Test Shape is:  (233153,) true


In [40]:
X_All, y_All = shuffle(X, y, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_All, y_All, test_size=0.33, random_state=42)
print('{} items total, {} true'.format(X_All.shape, y_All.shape))

(233153, 44) items total, (233153,) true


In [43]:
from xgboost import XGBClassifier
xgb1 = XGBClassifier( learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=42)

In [44]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

clf =xgb1 = XGBClassifier( learning_rate =0.01, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=42)

scores = cross_val_score(clf, X_All, y_All, scoring='accuracy', cv=10)
print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))

clf.fit(X_train, y_train)  # so that parts of the original pipeline are fitted

# evaluate(pipeline)

Accuracy: 0.784 ± 0.001


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=0.8)

In [45]:
booster = clf.get_booster()

In [49]:
booster.get_score()

{'ltv': 2458,
 'disbursed_amount': 1732,
 'State_ID': 857,
 'Perform_CNS_Scr': 1797,
 'Current_pincode_ID': 3398,
 'PRI_Sanction_Amt': 989,
 'PRI_No_Of_Acct': 954,
 'Employment.Type_1': 253,
 'No_Of_Inquiry': 823,
 'Deliquent_Acct_Last_Six_Mnth': 508,
 'Employee_ID': 1640,
 'asset_cost': 1387,
 'PRI_Overdue_Acct': 746,
 'Age': 1451,
 'Employment.Type_2': 340,
 'Credit_History_Len': 884,
 'PERFORM_CNS.SCORE.DESCRIPTION_5': 78,
 'PRI_Disbusrsed_Amt': 627,
 'VoterID_flag': 166,
 'manufacturer_id': 715,
 'Aadhar_flag': 78,
 'branch_id': 1249,
 'supplier_id': 1782,
 'PRI_Current_Bal': 884,
 'PRI_Act_Acct': 388,
 'SEC_Act_Acct': 32,
 'PRI_Instal_Amt': 1145,
 'SEC_Overdue_Acct': 58,
 'AVG_Acct_Age': 898,
 'PERFORM_CNS.SCORE.DESCRIPTION_2': 56,
 'SEC_Current_Bal': 100,
 'SEC_Instal_Amt': 119,
 'SEC_No_Of_Acct': 64,
 'SEC_Sanction_Amt': 74,
 'PERFORM_CNS.SCORE.DESCRIPTION_4': 57,
 'New_Acct_Last_Six_mnth': 279,
 'PERFORM_CNS.SCORE.DESCRIPTION_3': 49,
 'PAN_flag': 134,
 'SEC_Disbursed_Amt': 36,


In [68]:
from eli5 import show_weights, explain_weights_xgboost, explain_weights_df
weight_df=explain_weights_df(clf)

In [65]:
import eli5
from IPython.display import display
display(eli5.show_weights(clf))

Weight,Feature
0.0597,Perform_CNS_Scr
0.0589,ltv
0.0496,State_ID
0.0425,disbursed_amount
0.0413,No_Of_Inquiry
0.0391,PRI_Sanction_Amt
0.0380,PRI_Overdue_Acct
0.0351,VoterID_flag
0.0330,manufacturer_id
0.0325,Employment.Type_2


In [56]:
print(gini)

<IPython.core.display.HTML object>


In [62]:
from eli5 import explain_prediction, show_prediction
show_prediction(clf, X.iloc[1] , show_feature_values=True)#, show_feature_values=True)

In [69]:
weight_df.to_csv('Weight.csv', index=False)

In [70]:
weight_df

,feature,weight
0,Perform_CNS_Scr,0.059690
1,ltv,0.058885
2,State_ID,0.049604
3,disbursed_amount,0.042488
4,No_Of_Inquiry,0.041278
5,PRI_Sanction_Amt,0.039116
6,PRI_Overdue_Acct,0.038017
7,VoterID_flag,0.035123
8,manufacturer_id,0.033031
9,Employment.Type_2,0.032455
